<h1>GENERACIÓN DE PIPELINES</h1>

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

In [ ]:
iris = pd.read_csv("Iris.csv")

###Construcción del modelo

In [ ]:
iris.drop("Id", axis = 1)

SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm         Species
0              5.1           3.5            1.4           0.2     Iris-setosa
1              4.9           3.0            1.4           0.2     Iris-setosa
2              4.7           3.2            1.3           0.2     Iris-setosa
3              4.6           3.1            1.5           0.2     Iris-setosa
4              5.0           3.6            1.4           0.2     Iris-setosa
..             ...           ...            ...           ...             ...
145            6.7           3.0            5.2           2.3  Iris-virginica
146            6.3           2.5            5.0           1.9  Iris-virginica
147            6.5           3.0            5.2           2.0  Iris-virginica
148            6.2           3.4            5.4           2.3  Iris-virginica
149            5.9           3.0            5.1           1.8  Iris-virginica

[150 rows x 5 columns]

Separa variables predictoras de objetivo:

In [ ]:
X = iris[["Id","SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = iris["Species"]

1. Eliminamos la columna Id:

In [ ]:
X_ = X.drop("Id", axis = 1)

2. División en entrenamiento y test:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size = 0.25,
                                                    random_state = 123)

3. Estandarización de los datos:

In [ ]:
std = StandardScaler()
X_train_std = std.fit_transform(X_train)
X_test_std = std.transform(X_test)

4. PCA:

In [ ]:
pca = PCA(n_components = 2)
X_train_std_pca = pca.fit_transform(X_train_std)
X_test_std_pca = pca.transform(X_test_std)

5. Entrenamiento del modelo

In [ ]:
randomForest = RandomForestClassifier(criterion = "gini", max_depth= 3,
                               random_state = 123)
randomForest.fit(X_train_std_pca, y_train)

RandomForestClassifier(max_depth=3, random_state=123)

6. Predicción de mi modelo

In [ ]:
y_pred_train = randomForest.predict(X_train_std_pca)
y_pred_test = randomForest.predict(X_test_std_pca)

7. Cálculo de métricas de calidad del modelo:

In [ ]:
#TRAIN
accuracy_score(y_train, y_pred_train)

0.9553571428571429

In [ ]:
#TEST
accuracy_score(y_test, y_pred_test)

0.8421052631578947

###Construcción del pipeline:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.25,
                                                    random_state = 123)

Genera las funciones que incorporaremos a nuestro pipeline (junto con el modelo previamente definido)

In [ ]:
def eliminaColumna(X):
  X = X.drop("Id", axis = 1)
  return X

In [ ]:
transformer = FunctionTransformer(eliminaColumna)

In [ ]:
pca = PCA(n_components = 2)

In [ ]:
std = StandardScaler()

In [ ]:
pipe = Pipeline([('elimina_id', transformer),
                 ('pca', PCA(n_components = 2)),
                 ('std', std),
                 ('decision_tree', randomForest)], verbose = True)

Entrena pipeline con datos de train:

In [ ]:
pipe.fit(X_train, y_train)

[Pipeline] ........ (step 1 of 4) Processing elimina_id, total=   0.0s
[Pipeline] ............... (step 2 of 4) Processing pca, total=   0.0s
[Pipeline] ............... (step 3 of 4) Processing std, total=   0.0s
[Pipeline] ..... (step 4 of 4) Processing decision_tree, total=   0.1s


Pipeline(steps=[('elimina_id',
                 FunctionTransformer(func=<function eliminaColumna at 0x7fdf64cbb790>)),
                ('pca', PCA(n_components=2)), ('std', StandardScaler()),
                ('decision_tree',
                 RandomForestClassifier(max_depth=3, random_state=123))],
         verbose=True)

Predice:

In [ ]:
print(accuracy_score(y_test, pipe.predict(X_test)))

0.8947368421052632
